## 1.3 Introduction to Information Retrieval

Here we work with a data set scraped from eBay.  The data contains 9895 item titles and descriptions.

First we load the data - this is easiest with a `csv.reader`:

In [184]:
import csv
import re
import pandas as pd
from collections import Counter
from scipy.sparse import csr_matrix
from bokeh.plotting import figure, output_notebook, show
from bokeh.charts import Bar
from sys import getsizeof

In [185]:
with open("data/bike-items.txt") as f:
    r = csv.reader(f, delimiter=',', quotechar='"')
    rgx = re.compile(r'\b[a-zA-Z]+\b') 
    docs = [ (' '.join(re.findall(rgx, x[0])).lower(), ' '.join(re.findall(rgx, x[1])).lower())  for i,x in enumerate(r) if i > 1 ]

print('We have a list of (item title, description) tuples :\n + %s\n + %s' % (docs[0][0],docs[0][1]))

items_t = [ d[0] for d in docs ] # item titles
items_d = [ d[1] for d in docs ] # item descriptions
items_i = range(0, len(items_t)) # item id


We have a list of (item title, description) tuples :
 + cycling bicycle mtb bike fixie gloss carbon fiber riser bar handlebar
 + description feature easy to use made of high quality carbon fiber with the special design can save for a long time the carbon fiber handlebar is made of high quality carbon fiber so that you can use it relieved this quick disassembling carbon fiber handlebar is easy to use and one of the best gifts to your friends specification material carbon fiber color black handlebar clamp diameter mm length package included x cycling carbon fiber rise


Our raw data is in text form.  We need to convert it into a form more amenable to analysis.  In this notebok we look at ways of converting a collection of documents into a collection of vectors (a matrix).  To do this we need to *tokenize* the text - i.e. split it into words - and then create vectors of token frequency.  We will start by doing this the hard way and then look at how we can scale this up using scikit-learn.  Later on we will repeat this exercise using map-reduce.

We will proceed as follows:
1  Compute term frequency as a dictionary, a matrix and a sparse matrix
2  Implement a boolean search against the TF matrix
3  Introduce scikit-learn and the 'hashing-trick'
4  Compute TF.IDF for the set of documents

## Basic Term Frequency (TF) Matrix

Please note that this code is for understanding - it is not optimised or intended to scale!

Let's start with the first 10 item titles as out corpus:

In [186]:
corpus = items_t[0:5]
print(corpus)

['cycling bicycle mtb bike fixie gloss carbon fiber riser bar handlebar', 'bicycle rims x red speed internal hub wheel set beach cruiser bike', 'mavic crossride mountain bike wheels and wtb weirwolf tires', 'new kcnc arrow alloy stem black', 'rotor qxl aero oval road chainring']


### TF Dictionary 

Now we can compute the frequency of each term across the entire corpus:

In [187]:
tf = {}
for doc in corpus:
    for word in doc.split():
        if word in tf:
            tf[word] += 1
        else:
            tf[word] = 1

print(tf)

{'wtb': 1, 'qxl': 1, 'new': 1, 'carbon': 1, 'weirwolf': 1, 'black': 1, 'oval': 1, 'cruiser': 1, 'internal': 1, 'wheels': 1, 'speed': 1, 'red': 1, 'wheel': 1, 'crossride': 1, 'hub': 1, 'fixie': 1, 'beach': 1, 'and': 1, 'chainring': 1, 'stem': 1, 'fiber': 1, 'arrow': 1, 'bicycle': 2, 'mtb': 1, 'bar': 1, 'cycling': 1, 'bike': 3, 'set': 1, 'rims': 1, 'handlebar': 1, 'gloss': 1, 'mountain': 1, 'riser': 1, 'road': 1, 'tires': 1, 'aero': 1, 'kcnc': 1, 'alloy': 1, 'x': 1, 'mavic': 1, 'rotor': 1}


We can simplify by using a Counter rather than a dictionary:

In [188]:
tf = Counter()
for doc in corpus:
    for word in doc.split():
        tf[word] += 1
        
print(tf)

Counter({'bike': 3, 'bicycle': 2, 'wtb': 1, 'qxl': 1, 'new': 1, 'carbon': 1, 'weirwolf': 1, 'black': 1, 'oval': 1, 'cruiser': 1, 'internal': 1, 'wheels': 1, 'speed': 1, 'red': 1, 'wheel': 1, 'crossride': 1, 'hub': 1, 'fixie': 1, 'beach': 1, 'and': 1, 'chainring': 1, 'stem': 1, 'fiber': 1, 'arrow': 1, 'mtb': 1, 'bar': 1, 'cycling': 1, 'set': 1, 'rims': 1, 'handlebar': 1, 'gloss': 1, 'mountain': 1, 'riser': 1, 'road': 1, 'tires': 1, 'aero': 1, 'kcnc': 1, 'alloy': 1, 'x': 1, 'mavic': 1, 'rotor': 1})


No speed difference - but cleaner code:

In [189]:
def tf1(corpus):
    for doc in corpus:
        for word in doc.split(' '):
            if word in tf:
                tf[word] += 1
            else:
                tf[word] = 1        
    return tf

def tf2(corpus):
    tf = Counter()
    for doc in corpus:
        for word in doc.split(' '):
            tf[word] += 1
    return tf

%timeit tf1(corpus)
%timeit tf2(corpus)


10000 loops, best of 3: 24 µs per loop
10000 loops, best of 3: 24 µs per loop


### TF Matrix

Whilst the TF dictionary is a compact way to store the term frequency it is not much use for analysis.  We need a TF matrix where each document vector is the same length.  Now we convert the dictionary to a matrix:

In [190]:
def get_lexicon(corpus):
    lexicon = set()
    for doc in corpus:
        lexicon.update([word for word in doc.split()])
    return lexicon

lexicon = get_lexicon(corpus)

tfm =[]
for doc in corpus:
    for term in doc.split():
        tfv = [doc.split().count(word) for word in lexicon]
    tfm.append(tfv)
        
print([ x for x in tfm])

[[1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1], [1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]]


As number of terms increases this method becomes inefficient.  Here is a faster implementation:

In [191]:
def get_lexicon(corpus):
    lexicon = set()
    for doc in corpus:
        lexicon.update([word for word in doc.split()])
    return list(lexicon)

lexicon = get_lexicon(corpus)

tfm =[]
for doc in corpus:
    tfv = [0]*len(lexicon)
    for term in doc.split():
        tfv[lexicon.index(term)] += 1
    tfm.append(tfv)
        
print(tfm)

[[1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1], [1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]]


We can compare time for each:

In [192]:
def tfm1(corpus):
    
    def get_lexicon(corpus):
        lexicon = set()
        for doc in corpus:
            lexicon.update([word for word in doc.split()])
        return lexicon
    
    lexicon = get_lexicon(corpus)

    tfm =[]
    for doc in corpus:
        for term in doc.split():
            tfv = [doc.split().count(word) for word in lexicon]
        tfm.append(tfv)
    
    return tfm, lexicon

def tfm2(corpus):
    
    def get_lexicon(corpus):
        lexicon = set()
        for doc in corpus:
            lexicon.update([word for word in doc.split()])
        return list(lexicon)

    lexicon = get_lexicon(corpus)

    tfm =[]
    for doc in corpus:
        tfv = [0]*len(lexicon)
        for term in doc.split():
            tfv[lexicon.index(term)] += 1
        tfm.append(tfv)
    
    return (tfm, lexicon)

%timeit tfm1(corpus)
%timeit tfm2(corpus)


100 loops, best of 3: 1.13 ms per loop
10000 loops, best of 3: 33.3 µs per loop


In [194]:
# as size of corpus increases so does the sparsity

n = []
s = []
for i in range(100,1000,100):
    corpus = items_t[0:i]
    tfm, lexicon = tfm2(corpus)
    c =[ [x.count(0), x.count(1)] for x in tfm]
    n_zero = sum([ y[0] for y in c])
    n_one = sum([ y[1] for y in c])  
    s.append(float(n_one / (n_one + n_zero)))
    n.append(i)
    
output_notebook(hide_banner=True)
p = figure(x_axis_label='Documents', y_axis_label='Sparsity',
          plot_width=400, plot_height=400)
p.line(n, s, line_width=2)
p.circle(n, s, fill_color="white", size=8)
show(p)


We can take advantage of the sparsity and only store the non-zero elements of the TF matrix.

### Spare matrix storage

In [195]:
def tfm3(corpus):
    
    def get_lexicon(corpus):
        lexicon = set()
        for doc in corpus:
            lexicon.update([word for word in doc.split()])
        return list(lexicon)

    lexicon = get_lexicon(corpus)

    tfm =[]
    for doc_id, doc in enumerate(corpus):
        tfv = [0]*len(lexicon)
        for term in doc.split():
            tfv[lexicon.index(term)] += 1
        tfm.append([[(doc_id, t_id), t] for t_id, t in enumerate(tfv) if t > 0])
    
    return (tfm, lexicon)

tfm, lexicon = tfm3(corpus)
print(tfm[0])

[[(0, 71), 1], [(0, 206), 1], [(0, 230), 1], [(0, 349), 1], [(0, 551), 1], [(0, 606), 1], [(0, 666), 1], [(0, 945), 1], [(0, 1065), 1], [(0, 1395), 1], [(0, 1737), 1]]


We can also use compression to store this data even more efficiently - scikit-learn provides a compressed sparse matrix:

In [200]:
tfm=csr_matrix(tfm2(corpus)[0])
print(tfm[0,:])

l = ['tf2','tfm2','csr']
s = [getsizeof(tf2(corpus)[0]) , getsizeof(tfm2(corpus)[0]), getsizeof(csr_matrix(tfm2(corpus)[0]))]

df = pd.DataFrame({'Type':l, 'Size':s})

output_notebook(hide_banner=True)
p = Bar(df.sort_values(by='Size'), label='Type', values='Size',
        plot_width=400, plot_height=400)
show(p)

  (0, 71)	1
  (0, 206)	1
  (0, 230)	1
  (0, 349)	1
  (0, 551)	1
  (0, 606)	1
  (0, 666)	1
  (0, 945)	1
  (0, 1065)	1
  (0, 1395)	1
  (0, 1737)	1
['tf2', 'tfm2', 'csr'] [24, 7984, 56]


## Boolean Search

Now we have a tf matrix we can start to use it to find documents that contain words included in a query.  We will start by simply returning the documents from the corpus that match terms in our query:

In [260]:
def get_results1(qry, tfm, lexicon):
    qrv = [0]*len(lexicon)
    for term in qry.split():
        if term in lexicon:
            qrv[lexicon.index(term)] = 1

    results = []      
    for i, tfv in enumerate(tfm):
        score = sum([x[0] * x[1] for x in zip(tfv, qrv)])
        if score > 0:
               results.append([i, score])
    return results

tfm, lexicon = tfm2(items_t)
%timeit results = get_results1('front rear back led bike light', tfm , lexicon)
print([[r[1],items_t[r[0]]] for r in sorted(results, key=lambda t: t[1] * -1 )[:10]])

1 loops, best of 3: 5.39 s per loop
[[9892, 'bicycle rims x red speed internal hub wheel set beach cruiser bike'], [9891, 'bicycle rims x red speed internal hub wheel set beach cruiser bike'], [9890, 'bicycle rims x red speed internal hub wheel set beach cruiser bike'], [9889, 'bicycle rims x red speed internal hub wheel set beach cruiser bike'], [9888, 'bicycle rims x red speed internal hub wheel set beach cruiser bike'], [9887, 'bicycle rims x red speed internal hub wheel set beach cruiser bike'], [9886, 'bicycle rims x red speed internal hub wheel set beach cruiser bike'], [9885, 'bicycle rims x red speed internal hub wheel set beach cruiser bike'], [9884, 'bicycle rims x red speed internal hub wheel set beach cruiser bike'], [9883, 'yakima pack sks lock cores keys roof rack locking cylinders']]


But this is an expensive operation.  Each query has to be compared to all documents in the corpus.  We can speed this up by creating an inverted index:

In [224]:
def create_inverted_index(corpus):
    idx={}
    for i, doc in enumerate(corpus):
        for word in doc.split():
            if word in idx:
                idx[word].append(i)
            else:
                idx[word] = [i]
    return idx
            
idx = create_inverted_index(items_t)

print(set(idx['front']).intersection(set(idx['rear'])))
print(items_t[7676])


{512, 2049, 9216, 9733, 5131, 3597, 1039, 5648, 8212, 5140, 8729, 9753, 2075, 3611, 7708, 7198, 3618, 2597, 9768, 6697, 9257, 1582, 9263, 9264, 8242, 565, 2615, 6711, 6712, 8763, 4164, 581, 5701, 8269, 9295, 1107, 3156, 599, 3671, 9815, 7258, 8288, 609, 6244, 6247, 8807, 3181, 1135, 3697, 4722, 9841, 2676, 6772, 1651, 8824, 5241, 2682, 4219, 1660, 9341, 7805, 7806, 9344, 6274, 9346, 9859, 9860, 136, 5771, 7823, 9873, 1175, 3224, 9879, 9370, 1690, 9883, 7837, 8867, 5802, 2219, 2734, 9393, 7859, 7864, 1209, 699, 2749, 8896, 6344, 4809, 7881, 6859, 5330, 3795, 9429, 7893, 8408, 6361, 1254, 4839, 5351, 4331, 3310, 3822, 8437, 1783, 8954, 251, 7933, 2302, 9471, 3840, 5889, 4866, 9477, 5893, 6407, 7437, 1294, 4370, 6419, 7956, 1815, 6424, 8473, 794, 289, 7457, 1825, 1320, 9513, 5937, 4402, 8499, 4404, 6962, 5429, 7985, 6456, 7495, 9546, 4942, 2387, 2903, 9564, 1373, 1884, 9060, 1380, 1895, 8553, 2412, 3437, 9582, 8560, 6514, 4467, 9587, 7033, 8571, 1404, 9086, 8577, 9602, 4483, 905, 3977, 80

Now we just have to query for each of the terms and produce a set of results:

In [263]:
def get_results2(qry, idx):

    score = Counter()
    #for docs in [ idx[term] for term in qry.split()]:
    terms = qry.split()
    for term in terms:
        for doc in idx[term]:
            score[doc] += 1
            
    results=[]
    for x in [[r[0],r[1]] for r in zip(score.keys(), score.values())]:
        if x[1] > 0:
            # output [0] score, [1] doc_id
            results.append([x[1],x[0]])

    return results;
    

idx = create_inverted_index(items_t)
%timeit results = get_results2('front rear back led bike light', idx)
print([[r[0],items_t[r[1]]] for r in sorted(results, key=lambda t: t[0] * -1 )[:10]])



100 loops, best of 3: 5.01 ms per loop
[[8, 'frog waterproof bike light set led white front light led red rear light'], [7, 'bicycle bike led front head torch light led back rear tail flashlight lamp'], [6, 'ultra bright waterproof silicon led bicycle light set led front rear light'], [6, 'planet bike spok micro led front and back bike light set'], [6, 'waterproof white red led front head lamp led rear bike light set'], [5, 'modes cob led bicycle bike cycling front rear light usb rechargeable battery'], [5, 'x waterproof lamp bike bicycle front led head light rear safety flashlight'], [5, 'silicone bike bicycle cycling head front rear wheel led flash light lamp djnp'], [5, 'bicycle bike head front rear tail led light usb rechargeable lumens lo'], [5, 'silicone bike bicycle cycling head front rear wheel led flash light lamp buy']]


And we have a lot of documents that match on one of the words - bike is present in almost every document!

In [52]:
# histogram of score distribution in results